Implementating fem for truss using class

In [19]:
import math
import numpy as np
import copy

In [20]:
#import classes for nodes and elements
from node import node
from ele import ele

In [21]:
#define nodes along with boundary conditions
node1 = node(1,0,0,0,0,np.nan,np.nan)
node2 = node(2,2,0,0,0,np.nan,np.nan)
node3 = node(3,2,2,np.nan,np.nan,100000,-200000)

In [22]:
node1.print()
node2.print()
node3.print()

num =  1 
 pos_x = 0 
 pos_y =  0 
 dis_x =  0 
 dis_y =  0 
 f_x =  nan 
 f_y =  nan
num =  2 
 pos_x = 2 
 pos_y =  0 
 dis_x =  0 
 dis_y =  0 
 f_x =  nan 
 f_y =  nan
num =  3 
 pos_x = 2 
 pos_y =  2 
 dis_x =  nan 
 dis_y =  nan 
 f_x =  100000 
 f_y =  -200000


0

In [23]:
#define elements

ele1 = ele(1,1,1,1, 0 , node1 , node2)
ele2 = ele(2,1,1, 1 , math.pi / 2 , node2 , node3)
ele3 = ele(3,1*math.sqrt(2),1, 1 , math.pi / 4 , node1 , node3)


In [24]:
ele1.print()
ele2.print()
ele3.print()

num =  1 
 length= 1 
 area =  1 
 youngs modulus =  1 
 theta =  0 
 node_a =  <node.node object at 0x000001B7F426D7F0> 
 node_b =  <node.node object at 0x000001B7F426DE80>
num =  2 
 length= 1 
 area =  1 
 youngs modulus =  1 
 theta =  1.5707963267948966 
 node_a =  <node.node object at 0x000001B7F426DE80> 
 node_b =  <node.node object at 0x000001B7F426D6A0>
num =  3 
 length= 1.4142135623730951 
 area =  1 
 youngs modulus =  1 
 theta =  0.7853981633974483 
 node_a =  <node.node object at 0x000001B7F426D7F0> 
 node_b =  <node.node object at 0x000001B7F426D6A0>


0

In [25]:
#generating stiffness matrix for all elements

print(ele1.stiff())
print(ele2.stiff())
print(ele3.stiff())


[[ 1.  0. -1. -0.]
 [ 0.  0. -0. -0.]
 [-1. -0.  1.  0.]
 [-0. -0.  0.  0.]]
[[ 3.74939946e-33  6.12323400e-17 -3.74939946e-33 -6.12323400e-17]
 [ 6.12323400e-17  1.00000000e+00 -6.12323400e-17 -1.00000000e+00]
 [-3.74939946e-33 -6.12323400e-17  3.74939946e-33  6.12323400e-17]
 [-6.12323400e-17 -1.00000000e+00  6.12323400e-17  1.00000000e+00]]
[[ 0.35355339  0.35355339 -0.35355339 -0.35355339]
 [ 0.35355339  0.35355339 -0.35355339 -0.35355339]
 [-0.35355339 -0.35355339  0.35355339  0.35355339]
 [-0.35355339 -0.35355339  0.35355339  0.35355339]]


In [26]:
#list of nodes and elements
nodes = [node1 , node2 , node3]
eles = [ele1 , ele2 ,ele3]


In [27]:
#generate global stiffness matrix 

def globalstiff(eles , num_nodes):
    #dimension of global matrix
    dim = 2 * num_nodes
    #generate and store the element-wise stiffness matrices
    SMs = [e.stiff() for e in eles]
    
    GK = np.zeros((dim,dim))
    
    for e in eles:
        i = 2 * e.node_a.num -2
        j = 2 * e.node_a.num -1
        k = 2 * e.node_b.num -2
        l = 2 * e.node_b.num -1
        e_stiff = e.stiff()
        
        index = [i,j,k,l]
        index2d = [(a,b) for a in index for b in index]
        d = {i:0, j:1, k:2, l:3}
        for p,q in index2d:
            GK[p][q] = GK[p][q] + e_stiff[d[p]][d[q]]
            
    return GK
            
    

In [28]:
GK = globalstiff(eles , 3)
GK

array([[ 1.35355339e+00,  3.53553391e-01, -1.00000000e+00,
         0.00000000e+00, -3.53553391e-01, -3.53553391e-01],
       [ 3.53553391e-01,  3.53553391e-01,  0.00000000e+00,
         0.00000000e+00, -3.53553391e-01, -3.53553391e-01],
       [-1.00000000e+00,  0.00000000e+00,  1.00000000e+00,
         6.12323400e-17, -3.74939946e-33, -6.12323400e-17],
       [ 0.00000000e+00,  0.00000000e+00,  6.12323400e-17,
         1.00000000e+00, -6.12323400e-17, -1.00000000e+00],
       [-3.53553391e-01, -3.53553391e-01, -3.74939946e-33,
        -6.12323400e-17,  3.53553391e-01,  3.53553391e-01],
       [-3.53553391e-01, -3.53553391e-01, -6.12323400e-17,
        -1.00000000e+00,  3.53553391e-01,  1.35355339e+00]])

In [29]:
#create vector for node_dis and external_forces 

dis_list = []
for n in nodes:
    dis_list.append(n.dis_x)
    dis_list.append(n.dis_y)
    
dis = np.array(dis_list)
print(dis)

f_list = []
for n in nodes:
    f_list.append(n.f_x)
    f_list.append(n.f_y)
    
f = np.array(f_list)
print(f)


[ 0.  0.  0.  0. nan nan]
[     nan      nan      nan      nan  100000. -200000.]


In [30]:
#check for undetermined values in dis and create linear eqns

GK_dis =  copy.deepcopy(GK)
f_dis =  copy.deepcopy(f)

del_row = []
index_list = list(range(0,2*len(nodes)))
for i in range(0,2*len(nodes)):
    if dis_list[i] == 0:
        del_row.append(i)
        index_list.remove(i)
GK_dis = np.delete(GK_dis,del_row,0)
GK_dis = np.delete(GK_dis,del_row,1)
f_dis = np.delete(f_dis,del_row,0)
    
print(GK_dis)
print(f_dis)
print(index_list)
        
    
        

[[0.35355339 0.35355339]
 [0.35355339 1.35355339]]
[ 100000. -200000.]
[4, 5]


In [31]:
#check for undetermined values in force and create linear eqns

GK_dis =  copy.deepcopy(GK)
f_dis =  copy.deepcopy(f)

del_row = []
index_list = list(range(0,2*len(nodes)))
for i in range(0,2*len(nodes)):
    if dis_list[i] == 0:
        del_row.append(i)
        index_list.remove(i)
GK_dis = np.delete(GK_dis,del_row,0)
GK_dis = np.delete(GK_dis,del_row,1)
f_dis = np.delete(f_dis,del_row,0)
    
print(GK_dis)
print(f_dis)

[[0.35355339 0.35355339]
 [0.35355339 1.35355339]]
[ 100000. -200000.]


In [32]:
ans_dis = np.linalg.solve(GK_dis , f_dis)
ans_dis

array([ 582842.71247462, -300000.        ])

In [33]:
for i in range(0 , len(ans_dis)):
    dis[index_list[i]] = ans_dis[i]
dis

array([      0.        ,       0.        ,       0.        ,
             0.        ,  582842.71247462, -300000.        ])

In [34]:
ans_force = np.dot(GK,dis)
ans_force

array([-1.0000000e+05, -1.0000000e+05,  1.8369702e-11,  3.0000000e+05,
        1.0000000e+05, -2.0000000e+05])